In [98]:
from statsbombpy import sb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
import ast
import warnings
warnings.filterwarnings('ignore')
from sklearn.cluster import KMeans
from sklearn import preprocessing
import seaborn as sns

In [32]:
competitions = sb.competitions()
#Combining all 3 seasons into one dataframe
matches2021 = sb.matches(competition_id=37, season_id = 90) #a specific season of the womens FA Super League, also use season ID 90 (20-21), 42 (19-20),4(18-19)
matches1920 = sb.matches(competition_id=37, season_id = 42)
matches1819 = sb.matches(competition_id=37, season_id = 4)
frames = [matches1819, matches1920, matches2021]
FAWSL = pd.concat(frames)
#FAWSL = df.FAWSL


In [33]:
#print(FAWSL)
#events = sb.competition_events(home_team= 'Chelsea FCW', away_team= 'Arsenal WFC')


In [34]:
def findingallmatches(team1, team2, dataframe):
    '''
    findingallmatches('Manchester City WFC','Chelsea FCW', FAWSL)
    '''
    match_ids = []
    
    for index, row in dataframe.iterrows():
        # Check if the home team is team1 and the away team is team2
        if row['home_team'] == team1 and row['away_team'] == team2:
            # Print the match ID if the teams match
            match_id = row['match_id']
            match_ids.append(match_id)
        elif row['home_team'] == team2 and row['away_team'] == team1:
            # Otherwise, print a message indicating that the teams do not match
            match_id = row['match_id']
            match_ids.append(match_id)
    
    return match_ids


In [87]:
team1 = 'Arsenal WFC'
team2 = 'Chelsea FCW'
match_ids= findingallmatches(team1, team2, FAWSL)
print(match_ids)

[19736, 19785, 2275063, 2275090, 3775548, 3775627]


In [93]:
#create a df of all the matches from the ids
# this does not work
def findingtheevent(match_ids):
    events = []
    for i in range(len(match_ids)):
        match = sb.events(match_id = i)
    return events

In [95]:
#
#allmatches = findingtheevent(match_ids)

#print(sb.events(match_id=19736))

In [96]:
shots = allmatches[['match_id','shot_statsbomb_xg','timestamp', 'team', 'location', 'shot_outcome', 'shot_end_location']].dropna()

#print(shots)
shots['x1'] = shots.location.apply(lambda x: x[0])
shots['x2'] = shots.location.apply(lambda x: x[1])
shots['y1'] = shots.shot_end_location.apply(lambda x: x[0])
shots['y2']=  shots.shot_end_location.apply(lambda x: x[1])

shots['distance'] = np.sqrt(((shots['x2']-shots['x1']))**2+((shots['y2'])-(shots['y1'])))

T1_shots = shots.loc[shots['team']== team1]
T1_goal = T1_shots.loc[T1_shots['shot_outcome']=='Goal']
T1_miss = T1_shots.loc[T1_shots['shot_outcome']!='Goal']

T2_shots = shots.loc[shots['team']== team2]
T2_goal = T2_shots.loc[T2_shots['shot_outcome']=='Goal']
T2_miss = T2_shots.loc[T2_shots['shot_outcome']!='Goal']

In [119]:
#shots = shots.drop(shots.loc[(shots['x2']==0) & (shots['y2']==0)].index)
#shots = shots.drop(shots.loc[(shots['x2']==105) & (shots['y2']==68)].index)
#not sure why this is relevant


X = shots[['x1','x2','y1','y2','distance']]
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)


In [120]:
km = KMeans(n_clusters=10)
km = km.fit(X_scaled)

In [121]:
km.inertia_/len(shots)

0.49244064569070417

In [122]:
clusters = km.fit_predict(X_scaled)

In [123]:
shots ['clusters']= clusters

In [124]:
shots.to_csv('shots_clusters.csv',index = False)

In [125]:
shots = shots.sample(frac=1)

In [ ]:
pitch = Pitch(pitch_type='custom',half=False, goal_type='box', pitch_color='#f5f5f5', line_color='#696969' ,linewidth=2,pitch_length=105, pitch_width=68)
fig, axs = pitch.draw(nrows=10, ncols=5, figsize=(20,20))
for i in range(10):
    for j in range(5):
        cluster_analysis = shots.loc[shots['clusters']==(5*i + j)][:10]
        cluster_analysis.index = np.arange(0,3,1)
        axs[i,j].invert_yaxis()
        for k in range(len(cluster_analysis)):
                axs[i,j].plot([cluster_analysis['x1'][k],cluster_analysis['x2'][k]],[cluster_analysis['y1'][k],cluster_analysis['y2'][k]], color='blue')
        
        sns.scatterplot(data=cluster_analysis,x='x1',y='y1',color='blue',ax=axs[i,j])
        axs[i,j].set_title(str((5*i+j)))
plt.savefig('shot_clusters.png')